In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import re
import pandas as pd
import numpy as np

### Processing New, Mitochondria/Chloroplast Removed Data

In [2]:
leaf_1_dataset = False
leaf_2_dataset = False
soil_dataset = True

a, b, c = leaf_1_dataset, leaf_2_dataset, soil_dataset
if ((a and b) or (b and c) or (a and c)): 
    raise Exception('Cannot run two sets at the same time must only have 1 be true')
elif a == b == c:
    raise Exception('Need at least 1 set of data to be true')

In [7]:
if leaf_1_dataset:
    data_path = r"C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\original_data\leaf_microbiome_1\\"
    rel_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\leaf_microbiome_kingdom_to_strain\rerun_1_kingdom_to_strain\\'
    count_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\leaf_microbiome_count_k_to_spec\leaf_1_count_kingdom_to_strain\\'
    read_lengths = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data_Processing\read_lengths_folder\leaf_1_read_lengths_output.txt'
    
if leaf_2_dataset:
    data_path = r"C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\original_data\leaf_microbiome_2\\"
    rel_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\leaf_microbiome_kingdom_to_strain\rerun_2_kingdom_to_strain\\'
    count_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\leaf_microbiome_count_k_to_spec\leaf_2_count_kingdom_to_strain\\'
    read_lengths = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data_Processing\read_lengths_folder\leaf_2_read_lengths_output.txt'

if soil_dataset:
    data_path = r"C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\original_data\5488_soil_2\\"
    rel_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\soil_microbiome_kingom_to_strain\\'
    count_path = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data\soil_microbiome_count_k_to_spec\\'
    read_lengths = r'C:\Users\15404\Documents\GitHub\research_project\Final_Processing\Data_Processing\read_lengths_folder\5488_read_lengths_output.txt'

#### The purpose of this code is to create CSVs of the original data from superkingdom to species level taxonomy

In [8]:
file_list = [f for f in listdir(data_path) if isfile(join(data_path, f))]
# print(len(file_list))

In [9]:
taxon_list = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species'] #, 'strain']

### Create Relative Abundance Datasets

In [10]:
for taxon_level in taxon_list:
    df1 = pd.DataFrame(columns = ['sample', 'taxon_id', 'level', 'taxon_path', 'taxon', 'val'])
    for tsv in file_list:
        with open(data_path + tsv, 'r') as myfile:
            line_list = []
            for line in myfile:
                if line[0] != '@' and len(line) > 1:
                    fileContent = re.sub("\t", ",", line)
                    fileContent = re.sub("\n", "", fileContent)
                    new_line = fileContent.split(',')    
                    if new_line[1] == taxon_level and taxon_level != 'strain' and "||" not in line:
                        line_list.append(new_line)
                    elif new_line[1] == taxon_level and taxon_level == 'strain' and "||" not in line:
                        line_list.append(new_line[0:5])
        df2 = pd.DataFrame(line_list)
        df2.rename(columns = {0:'taxon_id', 1:'level', 2:'taxon_path', 3:'taxon', 4:'val'}, inplace = True) 
        df2.insert(0, "sample", [tsv]*len(line_list), True)
        
        df1 = pd.concat([df1, df2], ignore_index=True)
        
    print(df1.shape)
#     print(taxon_level)
#     print(max(df1['val']))
#     print(min(df1['val']))
    csv_name = 'rerun_soil_microbiome_' + taxon_level +'.csv'
    df1.to_csv(rel_path + csv_name)

(493, 6)
(6964, 6)
(9882, 6)
(19445, 6)
(40010, 6)
(129870, 6)
(546872, 6)


### Create Count Datasets

In [11]:
# Import read counts txt
#read_dict = {'JP_CMB_10_S135_L002_R1_001': 13299182, 'JP_CMB_11_S97_L002_R1_001': 19554716}

RL_dict = {}
read_lengths_file = open(read_lengths,"r")

rl_file_list = [line.split('\n') for line in read_lengths_file.readlines()]

for i in range(int(len(rl_file_list)/2)):
    RL_dict[rl_file_list[i*2][0][0:-6]] = rl_file_list[i*2 + 1][0]

In [12]:
for taxon_level in taxon_list:
    df1 = pd.DataFrame(columns = ['sample', 'taxon_id', 'level', 'taxon_path', 'taxon', 'val'])
    for tsv in file_list:
        with open(data_path + tsv, 'r') as myfile:
            line_list = []
            number_reads = float(RL_dict[tsv])
            for line in myfile:
                if line[0] != '@' and len(line) > 1:
                    fileContent = re.sub("\t", ",", line)
                    fileContent = re.sub("\n", "", fileContent)
                    new_line = fileContent.split(',')    
                    if new_line[1] == taxon_level and taxon_level != 'strain' and "||" not in line:
                        new_line[4] = float(new_line[4]) / 100 * number_reads
                        line_list.append(new_line)
                    elif new_line[1] == taxon_level and taxon_level == 'strain' and "||" not in line:
                        new_line[4] = float(new_line[4]) / 100 * number_reads
                        line_list.append(new_line[0:5])
        rel_ab_list = []
        for i in line_list:
            rel_ab_list.append(i[4])
        df2 = pd.DataFrame(line_list)
        df2.rename(columns = {0:'taxon_id', 1:'level', 2:'taxon_path', 3:'taxon', 4:'val'}, inplace = True) 
        df2.insert(0, "sample", [tsv]*len(line_list), True)
        
        df1 = pd.concat([df1, df2], ignore_index=True)
    
#     print(taxon_level)
    print(df1.shape)
#     print(taxon_level)
#     #print(df1.head())
#     print(max(df1['val']))
#     print(min(df1['val']))
#     print('')
    csv_name = 'count_soil_microbiome_' + taxon_level +'.csv'
    df1.to_csv(count_path + csv_name)
    #print(csv_name)

(493, 6)
(6964, 6)
(9882, 6)
(19445, 6)
(40010, 6)
(129870, 6)
(546872, 6)
